## Cargar de datos iniciales

In [1]:
import pandas as pd

# Cargar los datos desde el archivo Excel; las hojas 'Cupos' y 'Aspirantes' en DataFrames
file_path = 'informacion.xlsx'
cupos_programa = pd.read_excel(file_path, sheet_name='Cupos')
cupos_programa["LISTA_ESPERA"] = cupos_programa["CUPOS"].apply(lambda x: x+1 if x<15 else 15)
# cupos_programa["LISTA_ESPERA"] = cupos_programa["CUPOS"]
aspirantes_df = pd.read_excel(file_path, sheet_name='Aspirantes')

## Determinar puntajes diferenciales por prioridad de programas dentro de las opciones
  
   El puntaje diferencial es otorgado a las mujeres que eligieron un programa STEM y es equivalente a 4 puntos adicionales. Solo se aplicará para la obtención de un cupo en la opcion o prioridad donde se haya elegido un programa STEM. 

In [2]:
#Ordenar los aspirantes por puntaje de mayor a menor
aspirantes_df_sorted = aspirantes_df.sort_values(by='Puntaje', ascending=False)

In [3]:
#Definir diccionario con programas STEM
programas_stem = cupos_programa.set_index('PROGRAMAS')['STEM'].to_dict()

#Definir un diccionario para llevar la cuenta de los cupos disponibles por programa
cupos_disponibles_programa = cupos_programa.set_index('PROGRAMAS')['CUPOS'].to_dict()
cupos_disponibles_lista_espera = cupos_programa.set_index('PROGRAMAS')['LISTA_ESPERA'].to_dict()

In [4]:
#Calcular puntajes diferenciales por opción de programa (de 1 a 5)

#Definir función que realice el calculo siempre y cuendo el aspirante sea mujer y el programa en dicha
#opción sea STEM.

def calcular_puntaje_diferencial(row, nombre_opcion):
    # Verifica si el aspirante es mujer y si el programa es STEM
    es_mujer = row['Mujer'] == 1
    programa_es_stem = programas_stem.get(row[nombre_opcion], 0) == 1

    # Si ambas condiciones se cumplen, añade 4 al puntaje
    if es_mujer and programa_es_stem:
        return row['Puntaje'] + 4 #nota: Este 4 debe ser variable
    else:
        return row['Puntaje']

In [5]:
#Aplicar un bucle para que cree 5 columnas adicionales con puntajes diferenciales aplicando la función anterior.
for i in range(1, 6): #Nota este 6 debe ser variable 5+1 por tratarse de range
    opcion = f'Opción {i}'
    aspirantes_df[f'Puntaje Dif {opcion}'] = aspirantes_df.apply(
        lambda row: calcular_puntaje_diferencial(row, opcion), axis=1
    )

## Ordenar de mayor a menor los nuevos puntajes diferenciales en 5 df distintos

In [7]:
resultados_sorted = {}

for i in range(1, 6):
    # Nombres de las columnas relevantes
    opcion_col = f"Opción {i}"
    puntaje_dif_col = f"Puntaje Dif {opcion_col}"

    # Selecciona las columnas relevantes
    puntaje_opcion = aspirantes_df[["ID", "Puntaje", "Mujer", opcion_col, puntaje_dif_col]]

    # Ordena el DataFrame basado en el puntaje diferencial
    puntaje_opcion_sorted = puntaje_opcion.sort_values(by=puntaje_dif_col, ascending=False)

    # Guarda el DataFrame ordenado en un diccionario
    resultados_sorted[opcion_col] = puntaje_opcion_sorted

## Preparar el proceso de seleccion de elegibles y listas de espera

In [8]:
#Inicializar listas para los admitidos y las listas de espera por opción
elegibles_por_opcion = {}
lista_espera_por_opcion = {}

for i in range(1, 6):
    # Creando las listas vacías para cada opción
    elegibles_por_opcion[f'Opción {i}'] = []
    lista_espera_por_opcion[f'Opción {i}'] = []

In [9]:
asignados = set()

def asignar_cupos_elegibles(aspirante, preferencias, elegibles_por_opcion_n, asignados):
    if aspirante['ID'] in asignados:
        return False  # El aspirante ya ha sido asignado, no asignar de nuevo

    for opcion in preferencias:
        if cupos_disponibles_programa[opcion] > 0:
            # Admitir al aspirante en el programa de esta opción
            cupos_disponibles_programa[opcion] -= 1
            elegibles_por_opcion_n.append((aspirante['ID'], opcion))
            asignados.add(aspirante['ID'])  # Registrar al aspirante como asignado
            return True

    return False



def asignar_cupos_lista_espera(aspirante, preferencias, lista_espera_por_opcion_n, asignados):
    if aspirante['ID'] in asignados:
        return False  # El aspirante ya ha sido asignado, no asignar de nuevo

    for opcion in preferencias:
        if cupos_disponibles_lista_espera[opcion] > 0:
            # Admitir al aspirante en el programa de esta opción
            cupos_disponibles_lista_espera[opcion] -= 1
            lista_espera_por_opcion_n.append((aspirante['ID'], opcion))
            asignados.add(aspirante['ID'])  # Registrar al aspirante como asignado
            return True

    return False

## Seleccionar los elegibles

In [10]:
# Procesar cada aspirante para intentar asignarle un cupo en un programa de sus 5 opciones
# Asegurándose de que los diccionarios están inicializados
elegibles_por_opcion = {f'Opción {i}': [] for i in range(1, 6)}
        
for opcion in elegibles_por_opcion:
    for _, aspirante in resultados_sorted[opcion].iterrows():
        preferencias = [aspirante[opcion]]
        asignar_cupos_elegibles(aspirante, preferencias, elegibles_por_opcion[opcion], asignados)

## Seleccionar aspirantes que harán parte de la lista de espera

In [11]:
# Procesar cada aspirante para intentar asignarle un cupo en la lista de espera de alguno de sus 5 opciones
# Asegurándose de que los diccionarios están inicializados
lista_espera_por_opcion = {f'Opción {i}': [] for i in range(1, 6)}
        
for opcion in lista_espera_por_opcion:
    for _, aspirante in resultados_sorted[opcion].iterrows():
        preferencias = [aspirante[opcion]]
        asignar_cupos_lista_espera(aspirante, preferencias, lista_espera_por_opcion[opcion], asignados)

## Consolidar resultados

In [12]:
# A partir de los diccionarios cupos_disponibles_programa, elegibles_por_opcion, lista_espera_por_opcion crear un df

# Crear un diccionario para consolidar la información por programa
datos_por_programa = {programa: [] for programa in cupos_disponibles_programa.keys()}

# Función para agregar aspirantes a datos_por_programa
def agregar_aspirantes(opciones_dict):
    for opcion, aspirantes in opciones_dict.items():
        for aspirante_id, programa in aspirantes:
            datos_por_programa[programa].append(aspirante_id)

# Agregar elegibles y lista de espera a datos_por_programa
agregar_aspirantes(elegibles_por_opcion)
agregar_aspirantes(lista_espera_por_opcion)

# Crear el DataFrame usando el orden de los programas en cupos_disponibles_programa
df_programas = pd.DataFrame(dict([(programa, pd.Series(datos)) for programa, datos in datos_por_programa.items()]))

df_programas


,Programa 1,Programa 2,Programa 3,Programa 4,Programa 5,Programa 6,Programa 7,Programa 8,Programa 9,Programa 10
0,Aspirante_83,Aspirante_147,Aspirante_41,Aspirante_37,Aspirante_6,Aspirante_13,Aspirante_115,Aspirante_80,Aspirante_161,Aspirante_124
1,Aspirante_125,Aspirante_65,Aspirante_30,Aspirante_170,Aspirante_146,Aspirante_120,Aspirante_174,Aspirante_153,Aspirante_112,Aspirante_60
2,Aspirante_93,Aspirante_18,Aspirante_171,Aspirante_89,Aspirante_139,Aspirante_107,Aspirante_157,Aspirante_172,Aspirante_74,Aspirante_152
3,NaN,Aspirante_94,Aspirante_73,Aspirante_49,Aspirante_9,Aspirante_100,Aspirante_61,Aspirante_151,Aspirante_81,Aspirante_141
4,NaN,Aspirante_175,Aspirante_14,Aspirante_51,Aspirante_148,Aspirante_84,Aspirante_167,Aspirante_129,Aspirante_33,Aspirante_64
5,NaN,NaN,Aspirante_54,Aspirante_46,Aspirante_180,Aspirante_121,Aspirante_79,Aspirante_119,Aspirante_11,Aspirante_50
6,NaN,NaN,Aspirante_199,Aspirante_26,Aspirante_137,Aspirante_15,Aspirante_35,Aspirante_40,Aspirante_1,Aspirante_128
7,NaN,NaN,NaN,Aspirante_155,Aspirante_98,Aspirante_165,Aspirante_5,Aspirante_39,Aspirante_72,Aspirante_123
8,NaN,NaN,NaN,Aspirante_97,Aspirante_184,Aspirante_76,Aspirante_193,Aspirante_3,Aspirante_183,Aspirante_114
9,NaN,NaN,NaN,NaN,Aspirante_110,Aspirante_70,Aspirante_190,Aspirante_192,Aspirante_78,Aspirante_108


## Guardar puntajes diferenciales y resultados en archivos de excel

In [14]:
df_programas.to_excel("resultado_final.xlsx")
aspirantes_df.to_excel("puntajes_diferenciales.xlsx")